## Imports

In [6]:
import os
import h5py
import numpy as np
from pathlib import Path

import matplotlib.pyplot as plt

import keras
from keras.models import Sequential
from keras.layers import Conv1D, LSTM, Dropout, BatchNormalization, Dense, GlobalAveragePooling1D, TimeDistributed, MaxPooling1D, Flatten
from keras.optimizers import Nadam
from keras.callbacks import EarlyStopping
from keras.utils import to_categorical
from keras.regularizers import l2
from keras.optimizers.schedules import ExponentialDecay

from sklearn.utils import shuffle

from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

from parse_data import load_intra_train, load_intra_test, load_cross_train, load_cross_test

## Constants

In [7]:
DOWNSAMPLING_STEP = 8
NUM_CLASSES = 4
BATCH_SIZE = 3

ENCODE_MAP = {
        'rest': 0,
        'motor': 1,
        'memory': 2,
        'math': 3,
    }

INTRA_OR_CROSS_MODE = "cross"

if INTRA_OR_CROSS_MODE == "cross":
    load_train = load_cross_train
    load_test = load_cross_test
elif INTRA_OR_CROSS_MODE == "intra":
    load_train = load_intra_train
    load_test = load_intra_test


## Load data

In [8]:
filenamepath = ("C:/Users/Collin/Documents/Universiteit Utrecht/Periode 4/Deep learning/Project assignment/Final Project data/Final Project data/Intra/train/")

X_train, y_train = load_train(batch_size=BATCH_SIZE,downsample_step=DOWNSAMPLING_STEP,logs=True)

X.shape from Data\Cross\train: (94976, 3, 248)


## LSTM Model only

In [9]:
time_steps = X_train.shape[1]
num_channels = X_train.shape[2]

if y_train.ndim == 1:
    y_train = to_categorical(y_train, num_classes=NUM_CLASSES)

early_stop = EarlyStopping(
    monitor='val_loss',#'val_accuracy',#
    patience=3,
    restore_best_weights=True
)

# Load and preprocess test data
X_test, y_test = load_test(batch_size=BATCH_SIZE,downsample_step=DOWNSAMPLING_STEP,logs=1)

# One-hot encode labels
y_test = to_categorical(y_test, num_classes=4)


def run_model(learningrate, decaysteps, decayrate, epochs, batchsize):
    lstm_model = Sequential([
        TimeDistributed(Dense(16, kernel_regularizer=l2(1e-4)), input_shape=(time_steps, num_channels)),

        LSTM(16, return_sequences=True, kernel_regularizer=l2(1e-4)),
        BatchNormalization(),
        Dropout(0.3),

        LSTM(16, kernel_regularizer=l2(1e-3)),
        BatchNormalization(),
        Dropout(0.3),

        Dense(16, activation='relu', kernel_regularizer=l2(1e-4)),
        Dropout(0.3),

        Dense(NUM_CLASSES, activation='softmax')
    ])


    lstm_model.compile(optimizer=Nadam(ExponentialDecay(
        initial_learning_rate=learningrate,
        decay_steps=decaysteps,
        decay_rate=decayrate)),#0.94
                    loss='categorical_crossentropy',
                    metrics=['accuracy'])
    history = lstm_model.fit(
        X_train, y_train,   
        epochs=epochs,
        batch_size=batchsize,#beste tot nu toe: 64,
        validation_split=0.5,
        callbacks=[early_stop],
        verbose=0
    )
    # Evaluate
    test_loss, test_acc = lstm_model.evaluate(X_test, y_test, verbose=0)
    test_accuracy = test_acc
    return test_accuracy

Loading from test1
X.shape from Data\Cross\test1: (23744, 3, 248)
Loading from test2
X.shape from Data\Cross\test2: (23744, 3, 248)
Loading from test3
X.shape from Data\Cross\test3: (23744, 3, 248)


In [10]:
import itertools

learning_rates = [0.1,0.05,0.01,0.008]
decay_steps = [14,24,100,1000]
decay_rates = [0.2,0.5,0.8,0.9]
epochs = [2,4,6,8,10,12,14]
batch_sizes = [6,32,64,128]


all_possible_combinations = list(itertools.product(learning_rates, decay_steps, decay_rates, epochs, batch_sizes))
print(len(all_possible_combinations))

lengte = (len(all_possible_combinations))
previous_accuracy = 0.72
#0.01 100 0.8 8 32
best_lr = 0.008
best_ds = 100
best_dr = 0.8
best_ep = 8
best_bs = 32

for rate in learning_rates:
    accuracy1 = (run_model(learningrate=rate, decaysteps=best_ds, decayrate=best_dr, epochs=best_ep, batchsize=best_bs))
    accuracy2 = (run_model(learningrate=rate, decaysteps=best_ds, decayrate=best_dr, epochs=best_ep, batchsize=best_bs))
    accuracy3 = (run_model(learningrate=rate, decaysteps=best_ds, decayrate=best_dr, epochs=best_ep, batchsize=best_bs))
    accuracy = (accuracy1+accuracy2+accuracy3)/3
    if accuracy > previous_accuracy:
        previous_accuracy = accuracy
        best_lr = rate
print(best_lr)
for rate in decay_steps:
    accuracy1 = (run_model(learningrate=best_lr, decaysteps=rate, decayrate=best_dr, epochs=best_ep, batchsize=best_bs))
    accuracy2 = (run_model(learningrate=best_lr, decaysteps=rate, decayrate=best_dr, epochs=best_ep, batchsize=best_bs))
    accuracy3 = (run_model(learningrate=best_lr, decaysteps=rate, decayrate=best_dr, epochs=best_ep, batchsize=best_bs))
    accuracy = (accuracy1+accuracy2+accuracy3)/3
    if accuracy > previous_accuracy:
        previous_accuracy = accuracy
        best_ds = rate    
print(best_ds)    
for rate in decay_rates:
    accuracy1 = (run_model(learningrate=best_lr, decaysteps=best_ds, decayrate=rate, epochs=best_ep, batchsize=best_bs))
    accuracy2 = (run_model(learningrate=best_lr, decaysteps=best_ds, decayrate=rate, epochs=best_ep, batchsize=best_bs))
    accuracy3 = (run_model(learningrate=best_lr, decaysteps=best_ds, decayrate=rate, epochs=best_ep, batchsize=best_bs))
    accuracy = (accuracy1+accuracy2+accuracy3)/3
    if accuracy > previous_accuracy:
        previous_accuracy = accuracy
        best_dr = rate    
print(best_dr)
for rate in epochs:
    accuracy1 = (run_model(learningrate=best_lr, decaysteps=best_ds, decayrate=best_dr, epochs=rate, batchsize=best_bs))
    accuracy2 = (run_model(learningrate=best_lr, decaysteps=best_ds, decayrate=best_dr, epochs=rate, batchsize=best_bs))
    accuracy3 = (run_model(learningrate=best_lr, decaysteps=best_ds, decayrate=best_dr, epochs=rate, batchsize=best_bs))
    accuracy = (accuracy1+accuracy2+accuracy3)/3
    if accuracy > previous_accuracy:
        previous_accuracy = accuracy
        best_ep = rate    
print(best_ep)
for rate in batch_sizes:
    accuracy1 = (run_model(learningrate=best_lr, decaysteps=best_ds, decayrate=best_dr, epochs=best_ep, batchsize=rate))
    accuracy2 = (run_model(learningrate=best_lr, decaysteps=best_ds, decayrate=best_dr, epochs=best_ep, batchsize=rate))
    accuracy3 = (run_model(learningrate=best_lr, decaysteps=best_ds, decayrate=best_dr, epochs=best_ep, batchsize=rate))
    accuracy = (accuracy1+accuracy2+accuracy3)/3
    if accuracy > previous_accuracy:
        previous_accuracy = accuracy
        best_bs = rate    
print(best_bs)

1792
0.01
100
0.8
4
32


In [ ]:
# Fine tuning last bits

learning_rates = [0.01]
decay_steps = [90,95,100,105]
decay_rates = [0.8]
epochs = [4,5,6,7,8]
batch_sizes = [18,32,64]


all_possible_combinations = list(itertools.product(learning_rates, decay_steps, decay_rates, epochs, batch_sizes))
print(len(all_possible_combinations))

lengte = (len(all_possible_combinations))
previous_accuracy = 0.72
#0.01 100 0.8 8 32
best_lr = 0.01
best_ds = 100
best_dr = 0.8
best_ep = 8
best_bs = 32


print(best_lr)
for rate in decay_steps:
    accuracy1 = (run_model(learningrate=best_lr, decaysteps=rate, decayrate=best_dr, epochs=best_ep, batchsize=best_bs))
    accuracy2 = (run_model(learningrate=best_lr, decaysteps=rate, decayrate=best_dr, epochs=best_ep, batchsize=best_bs))
    accuracy3 = (run_model(learningrate=best_lr, decaysteps=rate, decayrate=best_dr, epochs=best_ep, batchsize=best_bs))
    accuracy = (accuracy1+accuracy2+accuracy3)/3
    if accuracy > previous_accuracy:
        previous_accuracy = accuracy
        best_ds = rate    
print(best_ds)    
for rate in decay_rates:
    accuracy1 = (run_model(learningrate=best_lr, decaysteps=best_ds, decayrate=rate, epochs=best_ep, batchsize=best_bs))
    accuracy2 = (run_model(learningrate=best_lr, decaysteps=best_ds, decayrate=rate, epochs=best_ep, batchsize=best_bs))
    accuracy3 = (run_model(learningrate=best_lr, decaysteps=best_ds, decayrate=rate, epochs=best_ep, batchsize=best_bs))
    accuracy = (accuracy1+accuracy2+accuracy3)/3
    if accuracy > previous_accuracy:
        previous_accuracy = accuracy
        best_dr = rate    
print(best_dr)
for rate in epochs:
    accuracy1 = (run_model(learningrate=best_lr, decaysteps=best_ds, decayrate=best_dr, epochs=rate, batchsize=best_bs))
    accuracy2 = (run_model(learningrate=best_lr, decaysteps=best_ds, decayrate=best_dr, epochs=rate, batchsize=best_bs))
    accuracy3 = (run_model(learningrate=best_lr, decaysteps=best_ds, decayrate=best_dr, epochs=rate, batchsize=best_bs))
    accuracy = (accuracy1+accuracy2+accuracy3)/3
    if accuracy > previous_accuracy:
        previous_accuracy = accuracy
        best_ep = rate    
print(best_ep)
for rate in batch_sizes:
    accuracy1 = (run_model(learningrate=best_lr, decaysteps=best_ds, decayrate=best_dr, epochs=best_ep, batchsize=rate))
    accuracy2 = (run_model(learningrate=best_lr, decaysteps=best_ds, decayrate=best_dr, epochs=best_ep, batchsize=rate))
    accuracy3 = (run_model(learningrate=best_lr, decaysteps=best_ds, decayrate=best_dr, epochs=best_ep, batchsize=rate))
    accuracy = (accuracy1+accuracy2+accuracy3)/3
    if accuracy > previous_accuracy:
        previous_accuracy = accuracy
        best_bs = rate    
print(best_bs)

60
0.01
105
0.8
6
32


In [15]:
print("best accuracy: ", previous_accuracy)
print(best_lr,
best_ds,
best_dr,
best_ep,
best_bs)

best accuracy:  0.7340895533561707
0.01 105 0.8 6 32
